# Text Generation
### Using the Base 200 Records

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import svm
from sklearn.metrics import average_precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
! pip install tensorflow
! pip install transformers
from transformers import pipeline, set_seed
import csv

In [ ]:
# Base 200, Pre-Processed
dfBase200 = pd.read_csv("/content/drive/Shareddrives/CS522 term project - bullying hatespeech/base200Processed.csv")


# this method takes a dataframe and generates data based on it
def generate_data(df_to_append, df_to_generate, rows_to_generate=200, len_of_generated_text=50):
  generator = pipeline('text-generation', model='gpt2')
  total_row_num = len(df_to_generate.index)
  cur_row = 0
 
  for i in range(rows_to_generate):
    # Reset the row number
    if cur_row >= total_row_num:
      cur_row = 0

    class_val = df_to_generate['class'][cur_row]
    len_of_cur_tweet = len(df_to_generate['tweet'][cur_row])
    gen = generator(df_to_generate['tweet'][cur_row], max_length=(len_of_generated_text), num_return_sequences=1)
    long_tweet = gen[0]['generated_text']

    # Get rid of original tweet
    new_tweet = long_tweet[len_of_cur_tweet:]
    new_row = {'class':class_val, 'tweet':new_tweet}
    df_to_append = df_to_append.append(new_row, ignore_index=True)

    cur_row = cur_row + 1

  return df_to_append


genData = generate_data(dfBase200, df_long_sample, 300, 50)
genData.to_csv('generated_data_with_long_tweets_500.csv')
genData

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

,Unnamed: 0,class,tweet
0,0.0,2,and we all fell asleep
1,1.0,2,J Cole ain't trash... Period.
2,2.0,2,"New show announced in Lutsen, MN at Papa Charl..."
3,3.0,0,"I'll do that, if you agree to take your Ethiop..."
4,4.0,2,"come on stay alive knights nation!!!!"" This is..."
...,...,...,...
495,NaN,2,You are a blue starburst.\n
496,NaN,2,The last time a politician went out of his wa...
497,NaN,2,.5 $30 XS2.75.\n\n
498,NaN,2,Now don't tell me you won't get sick of being...


# Preprocessing and saving the data that will be used for text generation


In [ ]:
processed_df = pd.read_csv("/content/drive/Shareddrives/CS522 term project - bullying hatespeech/all_trunc_data_processed.csv")
processed_df

# keep rows where tweet len is > l
l = 50
df_long = processed_df[processed_df['tweet'].str.len() >= l]
df_long

# keep rows where class is not 1
df_long = df_long[df_long['class']!= 1]

df_long.reset_index(drop=True)

df_long_sample = df_long.sample(200)
df_long_sample = df_long_sample.reset_index(drop=True)
df_long_sample

,Unnamed: 0,class,tweet
0,13090,2,After seeing Kansas play Duke a couple weeks a...
1,6677,2,Saw that there's a slope to the wire? That eno...
2,7480,2,Why do visiting teams WHINE about right field ...
3,5073,2,Was just handed this on my flight. Must be in ...
4,9695,2,NBC News projects Charlie Baker (R) wins Massa...
...,...,...,...
195,23301,0,"Bro everybody is a ho then, fuck it, your favo..."
196,20198,2,"HAHAHA ""chunky goalkeeper makes a save with hi..."
197,23206,0,On Nigger Day we honor nigger traditions by no...
198,5119,2,I wonder if the progs didn't relegate young bl...


# Preprocessing and Saving the Base 200

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/CS522 term project - bullying hatespeech/labeled_data_trunc.csv")
df = shuffle(df)

In [ ]:
# This block was for selec
hateSpeech = df[df['class'] != 1][3500:].sample(200)

hateSpeech = hateSpeech.reset_index(drop=True)
hateSpeech


,class,tweet
0,2,@Yankees and we all fell asleep @Frizzle18
1,2,RT @ShamarGotHoes: J Cole ain't trash... Period.
2,2,"New show announced in Lutsen, MN at Papa Charl..."
3,0,"@matt_chu22 @RyanRiehle I'll do that, if you a..."
4,2,"""@KnightfanNeal #UCFPINKPARTY come on stay ali..."
...,...,...
195,2,RT @ZitlalyL: It's okay chiquito &#128514; I'm...
196,2,I just put down a pan of brownies
197,2,#Yankees #FireCashman I don't want Arod back.
198,2,@SteveWorks4You @teaparty ...anti American Rep...


In [ ]:
hateSpeech[hateSpeech['class'] == 0].count()

class    55
tweet    55
dtype: int64

In [ ]:
# Pre-Processing before Data Generation

def preProcess(X):
  X = X.split()
  X = [x for x in X if not (x.startswith("@") or x.startswith('"@') or x.startswith('.@'))]
  X = [x for x in X if not x.startswith("http")]
  X = [x for x in X if not x.startswith("&")]
  X = [x for x in X if not x.startswith("#")]
  X = [x for x in X if not x=="RT"]
  X = [x for x in X if not ("&#" in x)]
  ret = ' '.join(X)
  return ret

In [ ]:
hateSpeech['tweet'] = hateSpeech['tweet'].apply(preProcess)
hateSpeech

dfBase200['tweet'] = dfBase200['tweet'].apply(preProcess)

NameError: ignored

In [ ]:
import csv

# hateSpeech.to_csv('genData500.csv')
hateSpeech.to_csv('base200Processed.csv')

hateSpeech

,class,tweet
0,2,and we all fell asleep
1,2,J Cole ain't trash... Period.
2,2,"New show announced in Lutsen, MN at Papa Charl..."
3,0,"I'll do that, if you agree to take your Ethiop..."
4,2,"come on stay alive knights nation!!!!"" This is..."
...,...,...
195,2,It's okay chiquito I'm not in the ghetto anymore
196,2,I just put down a pan of brownies
197,2,I don't want Arod back.
198,2,...anti American Republicans and teabaggers st...


In [ ]:
import csv

# hateSpeech.to_csv('genData500.csv')
hateSpeech[200:].to_csv('preProcessData_800.csv')

hateSpeech

In [ ]:
import csv

all_df.to_csv('all_trunc_data_processed.csv')

In [ ]:
import csv
import pandas as pd

rdf = pd.read_csv("/content/drive/Shareddrives/CS522 term project - bullying hatespeech/Datasets/preProcessed_Final_2000.csv")
print(rdf)
rdf = rdf.tail(1800)
s_rdf = rdf.sample(100)
s_rdf.to_csv("sample to check.csv")

      Unnamed: 0  index  class  ... Unnamed: 41 Unnamed: 42 Unnamed: 43
0              0   3571      2  ...         NaN         NaN         NaN
1              1  15713      2  ...         NaN         NaN         NaN
2              2  14211      2  ...         NaN         NaN         NaN
3              3  17955      2  ...         NaN         NaN         NaN
4              4  18087      0  ...         NaN         NaN         NaN
...          ...    ...    ...  ...         ...         ...         ...
1996        1996  24979      2  ...         NaN         NaN         NaN
1997        1997  24980      2  ...         NaN         NaN         NaN
1998        1998  24981      2  ...         NaN         NaN         NaN
1999        1999  24982      2  ...         NaN         NaN         NaN
2000        2000  24983      2  ...         NaN         NaN         NaN

[2001 rows x 44 columns]


In [ ]:
print (s_rdf)

      Unnamed: 0  index  class  ... Unnamed: 41 Unnamed: 42 Unnamed: 43
1264        1264  25848      2  ...         NaN         NaN         NaN
513          513  25097      2  ...         NaN         NaN         NaN
1298        1298  25882      0  ...         NaN         NaN         NaN
205          205  24789      2  ...         NaN         NaN         NaN
292          292  24876      0  ...         NaN         NaN         NaN
...          ...    ...    ...  ...         ...         ...         ...
637          637  25221      2  ...         NaN         NaN         NaN
1089        1089  25673      2  ...         NaN         NaN         NaN
773          773  25357      2  ...         NaN         NaN         NaN
233          233  24817      2  ...         NaN         NaN         NaN
1975        1975  24958      2  ...         NaN         NaN         NaN

[100 rows x 44 columns]


In [2]:
import csv
import pandas as pd

df = pd.read_csv("/content/drive/Shareddrives/CS522 term project - bullying hatespeech/hate_speech_labeled_data.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [3]:
df.groupby('class').count()

,Unnamed: 0,count,hate_speech,offensive_language,neither,tweet
class,,,,,,
0,1430,1430,1430,1430,1430,1430
1,19190,19190,19190,19190,19190,19190
2,4163,4163,4163,4163,4163,4163
